# Chi-square

- A statistical test to determine the difference between expected frequencies and observed frequencies in one or more categories of a contingency table. 

## Steps
Tinanic survival example: 

Null hypothesis: gender does not indicate survival. 

1. Count the number of people from each gender who survived and didn't. 

.          | Male | Female | Expected Frequency |
---        | ---  |    --- |                --- | 
Survived=1 |  161 |    337 |                498 |
Survived=0 |  681 |    127 |                808 |
Total      |  842 |    464 |               1306 |


2. Divide each count by the total people from each gener. 

.          | Male | Female | Expected Frequency |
---        | ---  |    --- |                --- | 
Survived=1 | 0.19 |   0.73 |               0.38 |
Survived=0 | 0.81 |   0.27 |               0.62 |

3. Calculate the expected frequency. Male survival rate = female survival rate = expected frequency if the null hypothesis is true. If not, we reject the null hypothesis. 


4. Calculate the statistic: 

$$
X^2 = \sum{\frac{(\text{Observed} - \text{Expected})^2}{\text{Expected}}}
$$

$$
X^2 = \frac{(0.19-0.38)^2}{0.38} + \frac{(0.73-0.38)^2}{0.38} + \frac{(0.81-0.62)^2}{0.62} + \frac{(0.27-0.62)^2}{0.62} 
$$


5. Calculate the degrees of freedom: 
(rows-1)(columns-1) = 1

6. Compare the statistic against a known distribution (Chi-square) and get P value. 



## When to use 
- Suited for categorical variables with categorical target.
- Variable values should be non-negative, and typically Boolean, frequencies, or counts. 



# Demo

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# to determine the chi2 value
from sklearn.feature_selection import chi2

# to select the features
from sklearn.feature_selection import SelectKBest

In [3]:
# load dataset

data = pd.read_csv('../datasets/titanic.csv')
data.shape

(1309, 14)

In [4]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [5]:
# the categorical variables in the titanic are pclass, sex and embarked

# first I will encode the labels of the categories into numbers

# for Sex / Gender
data['sex'] = np.where(data['sex'] == 'male', 1, 0)

# for Embarked
ordinal_label = {k: i for i, k in enumerate(data['embarked'].unique(), 0)}
data['embarked'] = data['embarked'].map(ordinal_label)


In [6]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[['pclass', 'sex', 'embarked']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((916, 3), (393, 3))

## Calculate chi-square p-value

In [7]:
# calculate the chi2 p_value between each of the variables
# and the target

# chi2 returns 2 arrays, one contains the F-Scores which are then
# evaluated against the chi2 distribution to obtain the pvalue.
# The pvalues are in the second array

f_score = chi2(X_train.fillna(0), y_train)

# the 2 arrays of values
f_score

(array([27.18283095, 95.93492132,  8.51621324]),
 array([1.85095118e-07, 1.18722647e-22, 3.51996172e-03]))

In [8]:
# 1) let's capture the p_values (in the second array, remember python indexes at 0) in a pandas Series
# 2) add the variable names in the index
# 3) order the variables based on their fscore

pvalues = pd.Series(f_score[1])
pvalues.index = X_train.columns
pvalues.sort_values(ascending=True)

sex         1.187226e-22
pclass      1.850951e-07
embarked    3.519962e-03
dtype: float64

The smaller the p value is, the more significant the feature is to predict the target. 

## Select top K features

In [9]:
sel_ = SelectKBest(chi2, k=1).fit(X_train, y_train)

# display features
X_train.columns[sel_.get_support()]

Index(['sex'], dtype='object')

In [10]:

# to remove the rest of the features:

X_train = sel_.transform(X_train)
X_test = sel_.transform(X_test)